In [1]:
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
import os

In [2]:
MAIN_DIR = os.getcwd()[:-len('scripts')]

In [3]:
theia_dir = MAIN_DIR+"dataset/gender/EnhancedUnbiasedDataset"
batch_size = 128

train_transform = transforms.Compose([transforms.Resize(226),
                                      transforms.RandomCrop(224),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

test_transform = transforms.Compose([transforms.Resize(226),
                                     transforms.RandomCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
theia_dataset = torchvision.datasets.ImageFolder(theia_dir, transform= train_transform)

test = int(len(theia_dataset)*0.3)  
valid = int(len(theia_dataset)*0.1)

test_data = torch.utils.data.Subset(theia_dataset, range(test)) 
test_data.dataset.transform = test_transform

valid_data = torch.utils.data.Subset(theia_dataset, range(test, test+valid))
valid_data.dataset.transform = test_transform

theia_data = torch.utils.data.Subset(theia_dataset, range(valid + test, len(theia_dataset)))

train_loader = torch.utils.data.DataLoader(theia_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size)

print(len(theia_data), len(test_data), len(valid_data))

3881 1939 646


In [4]:
import torchvision.models as models
num_classes = 2

model_resnet18 = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
model_resnet18.fc = nn.Sequential(nn.Linear(model_resnet18.fc.in_features,512),
                                  nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))
model_resnet18.load_state_dict(torch.load(MAIN_DIR+'checkpoints/gender/kaggle_model_best_96.pth.tar')['state_dict'])

for name, param in model_resnet18.named_parameters():
    if("bn" not in name):
        param.requires_grad = False


Using cache found in C:\Users\user/.cache\torch\hub\pytorch_vision_master


In [5]:

train_on_gpu = torch.cuda.is_available()

# if GPU is available, move the model to GPU
if train_on_gpu:
    model_resnet18 = model_resnet18.cuda()


In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model_resnet18.parameters(), lr=0.001)

In [ ]:
# number of epochs to train the model
n_epochs = 4
Train_loss = 0.0
valid_loss = 0.0
for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    ###################
    # train the model #
    ###################
    # model by default is set to train
    model_resnet18.train()
    for batch_i, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_resnet18(data)
        # calculate the batch loss
        
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss 
        train_loss += loss.item()
        
        if batch_i % 20 == 19:    # print training loss every specified number of mini-batches
            print('Epoch %d, Batch %d loss: %.16f' %
                  (epoch, batch_i + 1, train_loss / 20))
            Train_loss += train_loss/20
            train_loss = 0.0

        
        #/////////////////////     Validation        \\\\\\\\\\\\\\\\\\\
        
        model_resnet18.eval() # prep model for evaluation
        for data, target in valid_loader:
            # forward pass: compute predicted outputs by passing inputs to the model
            data, target = data.cuda(), target.cuda()
            output = model_resnet18(data)
            # calculate the loss
            loss = criterion(output, target)
            # update running validation loss 
            valid_loss += loss.item()


    # print training loss per epoch
    print('Epoch %d, training loss: %.10f  validation loss : %.10f' %
          (epoch, Train_loss, valid_loss/batch_i))
    Train_loss = 0.0
    valid_loss = 0.0

Epoch 1, Batch 20 loss: 0.6550118952989579
Epoch 1, Batch 40 loss: 0.5730834305286407


In [13]:
# Saving Model
torch.save(model_resnet18.state_dict(), 'checkpoints/gender/Enhanced_Model.pt')

In [49]:
test_loss = 0 
total = 0
correct = 0 
topk=(1,)
maxk = max(topk)
model_resnet18.eval()
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    #data = data.repeat(1, 3, 1, 1)
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_resnet18(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update  test loss 
    test_loss += loss.item()
    _, predicted = torch.topk(output, 1)
    total += target.size(0)
    correct += (predicted == target.view(predicted.shape)).sum().item()
    #correct += (predicted == target).sum().item()
# calculate avg test loss
test_loss = test_loss/batch_i
print('Test Loss: {:.6f}\n'.format(test_loss))
print('\nTest Accuracy (Overall): {:.3f}\n'.format(correct/total))

Test Loss: 0.119744


Test Accuracy (Overall): 0.861

